### Prepossessing

In [8]:
import numpy as np
import pandas as pd

df = pd.read_parquet('Metal_all_20180116.snappy.parquet')
# print ('***** Data Types *****' + '\n' + str(df.dtypes) + '\n\n' + 
#        '***** Unique Ligands *****' + '\n' + str(df.ligandId.unique()))

# Extract zinc-binded, single-chained protein sequences
df_zn = df.loc[df['ligandId'] == 'ZN']
df_zn_single = df_zn.loc[df_zn['interactingChains'] == 1]
seqs = np.array(df_zn_single.sequence)
target = np.array(df_zn_single.fingerprint)

In [21]:
print (df['ligandId'])

0        MN
1        CA
2        CA
3        CA
4        MG
5        ZN
6        ZN
7        ZN
8        CU
9        FE
10       MG
11       ZN
12       ZN
13       ZN
14       CA
15       CA
16       ZN
17       CA
18       CA
19       CA
20       CA
21       CA
22       CA
23       ZN
24       CA
25       MN
26       CA
27       ZN
28       FE
29       CA
         ..
76327    MN
76328    ZN
76329    ZN
76330    ZN
76331    ZN
76332    MN
76333    ZN
76334    ZN
76335    ZN
76336    MN
76337    MN
76338    NI
76339    ZN
76340    ZN
76341    ZN
76342    ZN
76343    ZN
76344    FE
76345    CA
76346    ZN
76347    ZN
76348    ZN
76349    ZN
76350    MG
76351    MG
76352    CA
76353    CA
76354    MG
76355    ZN
76356    ZN
Name: ligandId, Length: 76357, dtype: object


In [15]:
display(df[0:5])

,structureChainId,ligandId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100
0,1A0O.A,MN,"[11, 55, 57]","[13, 57, 59]",ADKELKFLVVDDFSTMRRIVRNLLKELGFNNVEEAEDGVDALNKLQ...,1,115.0,386.0,404.0,371.0,325.0,313.0,1621.0
1,1A25.A,CA,"[107, 110, 111, 113]","[248, 251, 252, 254]",GSPGISGGGGGILDSMERRGRIYIQAHIDREVLIVVVRDAKNLVPM...,2,632.0,5212.0,24455.0,28660.0,32138.0,33630.0,44596.0
2,1A25.A,CA,"[45, 46, 105, 107, 113]","[186, 187, 246, 248, 254]",GSPGISGGGGGILDSMERRGRIYIQAHIDREVLIVVVRDAKNLVPM...,2,632.0,5212.0,24455.0,28660.0,32138.0,33630.0,44596.0
3,1A25.A,CA,"[46, 52, 105, 106, 107]","[187, 193, 246, 247, 248]",GSPGISGGGGGILDSMERRGRIYIQAHIDREVLIVVVRDAKNLVPM...,1,632.0,5212.0,24455.0,28660.0,32138.0,33630.0,44596.0
4,1AJD.B,MG,"[50, 154, 321]","[51, 155, 322]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0


In [5]:
del df,df_zn,df_zn_single

In [ ]:
# Filepaths
dict_path = "./dictionaries/"
model_path = "./models/"
hist_path = "./histories/"
fig_path = "./figs/"

In [ ]:
# Load
seqs_dict = {}
with open(dict_path + "seq_n_gram_to_vec_dict_w_UX", 'r') as fp:
        seqs_dict = json.load(fp)

seqs_dict_onehot = {}
with open(dict_path + "seqs_dict_onehot", 'r') as fp:
        seqs_dict_onehot = json.load(fp)

#### Data Generator

In [3]:
params = {'dim': (32,32,32),
          'batch_size': 64,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

In [4]:
# params: input_shape, batch_size, shuffle, classes/labels
class myGenerator():
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1, n_classes=2, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        
    def __data_generation(self, list_IDs_temp):
        #   'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes) 
    def get_params(self):
        return {'dim': self.dim,
                'batch_size': self.batch_size,
                'n_classes': self.n_classes,
                'n_channels': self.n_channels,
                'shuffle': self.shuffle}

### Input

In [ ]:
class myInput():
    def _init_(self, generator):
        self.generator = generator
        self.params = generator.get_params()
        
    def get_params(self):
        return self.params

### Model

In [ ]:
model_params = {'num_of_inputs' : 1,
                'output_dim' : 706,
                'optimizer' : Adam(),
                'callback' : myCallback
                'fold_size' : 500,
                'epoch_size' : 10,
                'loss' : 'binary_crossentropy'}

In [ ]:
class myModel():
    def _init_(self, myInput, userModel, model_params = {}):
        self.config = myInput.get_config
        self.model_param = model_params
        self.userModel = userModel
        
    def create_model(self):
        
    def train(self):
        self.create_model()
        
    

### Evaluation